In [ ]:
import tensorflow as tf

# Check if TensorFlow can detect a GPU
gpu_device_name = tf.test.gpu_device_name()

if gpu_device_name:
    print('GPU device found:', gpu_device_name)
else:
    print("No GPU available. Using CPU instead.")

GPU device found: /device:GPU:0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
import tensorflow as tf
from tensorflow.keras import layers, models

# zero noise

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/AML/A1/df_syn_train_0_0_.csv")

In [ ]:
df = df.sample(frac = 1)

In [ ]:
# Separate features and target variable
X = df.drop(['era', 'target_10_val', 'target_5_val'], axis=1)
y = df['era']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Initialize the Random Forest classifier with desired hyperparameters
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the Random Forest classifier on the training data
rf_classifier.fit(X_train, y_train)

# Predictions on training set
train_predictions = rf_classifier.predict(X_train)

# Predictions on testing set
test_predictions = rf_classifier.predict(X_test)

# Calculate and print accuracy on training set
train_accuracy = accuracy_score(y_train, train_predictions)
print(f'Training Accuracy: {train_accuracy:.2f}')

# Calculate and print accuracy on testing set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f'Testing Accuracy: {test_accuracy:.2f}')


Training Accuracy: 0.99
Testing Accuracy: 0.96


# LOW NOISE

In [ ]:
df2 = pd.read_csv("/content/drive/MyDrive/AML/A1/df_synA_train_shuffled.csv")

In [ ]:
df2 = df2.sample(frac = 1)

In [ ]:
# Separate features and target variable
X = df2.drop(['era', 'target_10_val', 'target_5_val', 'data_type'], axis=1)
y = df2['era']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
X_train.shape

(249600, 26)

In [ ]:
X_test.shape

(62400, 26)

In [ ]:
# Initialize the Random Forest classifier with desired hyperparameters
rf_classifier2 = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the Random Forest classifier on the training data
rf_classifier2.fit(X_train, y_train)

# Predictions on training set
train_predictions = rf_classifier2.predict(X_train)

# Predictions on testing set
test_predictions = rf_classifier2.predict(X_test)

# Calculate and print accuracy on training set
train_accuracy = accuracy_score(y_train, train_predictions)
print(f'Training Accuracy: {train_accuracy:.2f}')

# Calculate and print accuracy on testing set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f'Testing Accuracy: {test_accuracy:.2f}')


Training Accuracy: 0.99
Testing Accuracy: 0.83


In [ ]:
# Initialize the Gradient Boosting classifier
gb_classifier2 = GradientBoostingClassifier(n_estimators=100, random_state=42)

# Train the Gradient Boosting classifier on the training data
gb_classifier2.fit(X_train, y_train)

# Predictions on training set
train_predictions = gb_classifier2.predict(X_train)

# Predictions on testing set
test_predictions = gb_classifier2.predict(X_test)

# Calculate and print accuracy on training set
train_accuracy = accuracy_score(y_train, train_predictions)
print(f'Training Accuracy: {train_accuracy:.2f}')

# Calculate and print accuracy on testing set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f'Testing Accuracy: {test_accuracy:.2f}')


Training Accuracy: 0.77
Testing Accuracy: 0.77


In [ ]:
# Define the noise-robustness attention mechanism
def attention_mechanism(input_dim):
    inputs = layers.Input(shape=(input_dim,))
    attention_scores = layers.Dense(input_dim, activation='softmax')(inputs)
    return models.Model(inputs=inputs, outputs=attention_scores)

# Load your dataset and preprocess it if necessary
# Assuming you have X_train, X_test, y_train, y_test loaded

# Initialize the MLPClassifier model
input_dim = X_train.shape[1]  # Number of features in the dataset
num_classes = len(np.unique(y_train))  # Number of classes in the target variable

mlp_classifier = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(input_dim,)),
    layers.Dense(150, activation='relu'),
    layers.Dense(100, activation='relu'),
    layers.Dense(50, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])
# Compile the model
mlp_classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model with gradient scaling using attention mechanism
def train_with_attention(X_train, y_train, X_test, y_test):
    # Initialize the attention mechanism
    attention_model = attention_mechanism(input_dim)

    # Train the attention mechanism
    attention_model.compile(optimizer='adam', loss='mse')
    attention_model.fit(X_train, X_train, epochs=10, batch_size=32, verbose=0)

    # Compute attention scores
    attention_scores = attention_model.predict(X_train)

    # Scale gradients based on attention scores
    scaled_gradients = X_train * attention_scores

    # Train MLP classifier with scaled gradients
    mlp_classifier.fit(scaled_gradients, y_train, epochs=10, batch_size=32, verbose=0)

# Call the train_with_attention function with your dataset
train_with_attention(X_train, y_train, X_test, y_test)

# Predictions on training set
train_predictions = np.argmax(mlp_classifier.predict(X_train), axis=1)

# Predictions on testing set
test_predictions = np.argmax(mlp_classifier.predict(X_test), axis=1)

# Calculate and print accuracy on training set
train_accuracy = accuracy_score(y_train, train_predictions)
print(f'Training Accuracy: {train_accuracy:.2f}')

# Calculate and print accuracy on testing set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f'Testing Accuracy: {test_accuracy:.2f}')


1950/1950 [==============================] - 4s 2ms/step
Training Accuracy: 0.08
Testing Accuracy: 0.08


# HIGH NOISE

In [ ]:
df3 = pd.read_csv("/content/drive/MyDrive/AML/A1/df_synA_test_hard_shuffled_sample.csv")

In [ ]:
df3 = df3.sample(frac = 1)

In [ ]:
df3.head()

,Open_n_val,High_n_val,Low_n_val,Close_n_val,Volume_n_val,SMA_10_val,SMA_20_val,CMO_14_val,High_n-Low_n_val,Open_n-Close_n_val,...,SMA_20-SMA_10_changelen_val,Close_n_slope_3_changelen_val,Close_n_slope_5_changelen_val,Close_n_slope_10_changelen_val,row_num,day,era,target_10_val,target_5_val,data_type
4887,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.25,1.00,...,0.25,1.00,1.00,1.00,89,535,7,0.00,0.00,validation
177849,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.50,0.75,...,0.50,0.00,0.50,0.00,126,484,23,0.25,0.25,validation
5846,0.25,0.25,0.50,0.50,0.0,0.25,0.75,0.5,0.00,0.00,...,1.00,1.00,1.00,0.75,120,387,17,1.00,1.00,validation
183195,1.00,1.00,1.00,0.75,0.0,1.00,1.00,0.0,0.50,1.00,...,0.75,0.25,0.25,0.25,103,464,16,0.75,0.75,validation
244626,0.00,0.25,0.25,0.25,0.0,0.00,0.00,0.5,0.50,0.25,...,1.00,1.00,0.50,1.00,112,374,16,0.25,1.00,validation


In [ ]:
df3.shape

(249600, 30)

In [ ]:
# Separate features and target variable
X = df3.drop(['era', 'target_10_val', 'target_5_val', 'data_type'], axis=1)
y = df3['era']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Initialize the Random Forest classifier with desired hyperparameters
rf_classifier3 = RandomForestClassifier(n_estimators=200, random_state=42)

# Train the Random Forest classifier on the training data
rf_classifier3.fit(X_train, y_train)

# Predictions on training set
train_predictions = rf_classifier3.predict(X_train)

# Predictions on testing set
test_predictions = rf_classifier3.predict(X_test)

# Calculate and print accuracy on training set
train_accuracy = accuracy_score(y_train, train_predictions)
print(f'Training Accuracy: {train_accuracy:.2f}')

# Calculate and print accuracy on testing set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f'Testing Accuracy: {test_accuracy:.2f}')


Training Accuracy: 1.00
Testing Accuracy: 0.59


In [ ]:
# Initialize the Gradient Boosting classifier
gb_classifier3 = GradientBoostingClassifier(n_estimators=100, random_state=42)

# Train the Gradient Boosting classifier on the training data
gb_classifier3.fit(X_train, y_train)

# Predictions on training set
train_predictions = gb_classifier3.predict(X_train)

# Predictions on testing set
test_predictions = gb_classifier3.predict(X_test)

# Calculate and print accuracy on training set
train_accuracy = accuracy_score(y_train, train_predictions)
print(f'Training Accuracy: {train_accuracy:.2f}')

# Calculate and print accuracy on testing set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f'Testing Accuracy: {test_accuracy:.2f}')


Training Accuracy: 0.53
Testing Accuracy: 0.52


In [ ]:
# Initialize the AdaBoost classifier
adaboost_classifier3 = AdaBoostClassifier(n_estimators=100, random_state=42)

# Train the AdaBoost classifier on the training data
adaboost_classifier3.fit(X_train, y_train)

# Predictions on training set
train_predictions = adaboost_classifier3.predict(X_train)

# Predictions on testing set
test_predictions = adaboost_classifier3.predict(X_test)

# Calculate and print accuracy on training set
train_accuracy = accuracy_score(y_train, train_predictions)
print(f'Training Accuracy: {train_accuracy:.2f}')

# Calculate and print accuracy on testing set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f'Testing Accuracy: {test_accuracy:.2f}')

Training Accuracy: 0.34
Testing Accuracy: 0.34
